# Importing libraries and loading prepared data

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from data_preparation import *
books, ratings, users, Y, R = prepared_data()

# Creating model parameters and prediction function

Let's set the initial number of dimensions of X, W and B parameters vectors to 10 in order to find a good balance between complexity and overfitting.


In X matrix, the i-th row corresponds to the feature vector for the book i. Similarly, in W matrix, the j-th row corresponds to the parameter vector for user j. The B vector corresponds to the user bias. Initially, let's set these values to random.

In [3]:
num_books, num_users = Y.shape
W = tf.Variable(np.random.normal(size=(num_users, 10)).astype(np.float32))
B = tf.Variable(np.random.normal(size=(num_users)).astype(np.float32))
X = tf.Variable(np.random.normal(size=(num_books, 10)).astype(np.float32))


Predicted rating is calculated with this pattern: $x^{(i)}$ ⋅ $w^{(j)}$ + $b^{(j)}$. We count the dot product of movie feature vector and user parameter vector W and we add the user bias B.

In [2]:
def predict(users, books):
    prediction = tf.reduce_sum(tf.gather(W, users) * tf.gather(X, books), axis=1)
    prediction += tf.gather(B, users)
    return prediction

# Cost function

The collaborative filtering cost function is given by adding sum of squarred errors and regularization terms.

In [ ]:
def collfilt_cost_func(Y):
    non_zero_ratings = np.nonzero(Y)
    users = non_zero_ratings[1]
    books = non_zero_ratings[0]
    ratings = Y[non_zero_ratings]

    pred = predict(users, books)
    cost = tf.reduce_mean(tf.square(pred - ratings))

    cost += tf.reduce_sum(W**2) + tf.reduce_sum(B**2) + tf.reduce_sum(X**2)

    return cost

# Mean normalization

Mean normalization makes the algorithm behave a lot better and faster. We normalize the ratings by computing the mean rating for each book and subtracting it from the ratings.

In [ ]:
mean_ratings = np.nanmean(Y, axis=1, keepdims=True)
Y_normalized = Y - mean_ratings

# Gradient descent

Let's use gradient descent to minimize the cost function. I will set the learning rate to 0.1.

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

# Train the model